In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head(10)

In [ ]:
df.shape



In [ ]:
df.describe()

In [ ]:
df['DEATH_EVENT'].value_counts()

In [ ]:
#checking for missing values
df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#checking correlation
df.corr()

In [ ]:
#heatmap for corr
sns.heatmap(df.corr(),cmap="YlGnBu")

## handling Outliers using percentile method

In [ ]:
#handling Outliers using percentile method

df.columns
mini=df['creatinine_phosphokinase'].quantile(0.02)
df[df['creatinine_phosphokinase']<mini]
maxi=df['creatinine_phosphokinase'].quantile(.978)
df[df['creatinine_phosphokinase']>maxi]

df2=df[(df['creatinine_phosphokinase']>=mini) & (df['creatinine_phosphokinase']<maxi)]
df2.sample(10)

In [ ]:
df2.describe()

# handling outliers using Z score or standard ditribution

In [ ]:
#handling outliers using Z score or standard ditribution
sns.distplot(df2['platelets'])

In [ ]:
mean=df2['platelets'].mean()
sd=df2['platelets'].std()
df2['zscore']=(df2.platelets-mean)/sd
df2.shape
df2 =df2[(df2['zscore']<3) & (df2['zscore']>-3)]
sns.distplot(df2['platelets'])
df2.shape


In [ ]:
plt.hist(df['serum_sodium'])

In [ ]:
sns.boxplot(y='serum_sodium',data=df2)

In [ ]:
plt.boxplot(df['serum_sodium'])

# IQR method to remove outliers

In [ ]:
Q1=df2['serum_sodium'].quantile(0.25)
Q3=df2['serum_sodium'].quantile(0.75)
iqr=Q3-Q1
lowerlimit=Q1-1.5*iqr
upperlimit=Q3+1.5*iqr


In [ ]:
df2=df2[(df2['serum_sodium']>lowerlimit) & (df2['serum_sodium']<upperlimit)]
df2


In [ ]:
X=df.iloc[:,:12]
y=df['DEATH_EVENT']

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model=ExtraTreesClassifier()
model.fit(X,y)

In [ ]:
imp=pd.Series(data=model.feature_importances_,index=X.columns)
imp.nlargest(5).plot(kind ='bar')

In [ ]:
sns.jointplot('time','ejection_fraction',data=df2,kind ='reg')
sns.pairplot(df2,hue='DEATH_EVENT')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,random_state=0)
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
#Randomized search CV


n_estimators=[int(i) for i in np.linspace(100,1200,12)]
max_features=['auto','sqrt','log2']
max_depth=[int(i) for i in np.linspace(5,30,6)]
min_samples_leaf=[1,2,5,10]

In [ ]:
#create random_grid
random_grid={'n_estimators':n_estimators,
            'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_leaf':min_samples_leaf}
random_grid

In [ ]:
rf=RandomForestClassifier()
random=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=10,cv=5,verbose=2,scoring='roc_auc',random_state=42,return_train_score=True,n_jobs=1)
random.fit(X_train,y_train)


In [ ]:
random.best_params_

In [ ]:
random.best_score_

In [ ]:
random.score(X_test,y_test)

In [ ]:
plt.figure()
plt.plot(df2[df2['DEATH_EVENT']==1]['time'],df2[df2['DEATH_EVENT']==1]['platelets'],'ro',label='yes')
plt.plot(df2[df2['DEATH_EVENT']==0]['time'],df2[df2['DEATH_EVENT']==0]['platelets'],'bo',label='no')
plt.legend()

In [ ]:
y_pred=random.predict(X_test)
y_pred

In [ ]:
sns.distplot(y_pred-y_test,bins=5)

#we get a very narrow normal distribution whuch shows are model is performimg very well

In [ ]:
plt.scatter(y_pred,y_test)

In [ ]:
import pickle

with open('random_forest_classifier.pkl','wb') as file:
    pickle.dump(random,file) #dumps the model as pickle file